In [1]:
from ros_edf.ros_interface import EdfRosInterface
from ros_edf.pc_utils import pcd_from_numpy, draw_geometry, reconstruct_surface
from edf.data import PointCloud, SE3, TargetPoseDemo, DemoSequence, save_demos
from edf.pc_utils import check_pcd_collision, optimize_pcd_collision

import torch
import numpy as np
import yaml
import plotly as pl
import plotly.express as ple
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def move_simple(target_poses: SE3, env_interface: EdfRosInterface) -> str:

    ###### Plan ######
    plan_targets = [(target_pose, "default", {}) for target_pose in target_poses]
                   
    plan_results, plans = env_interface.move_plans(targets=plan_targets)
    plan_success = plan_results[-1]
    if not plan_success:
        return 'PLAN_FAIL'
    
    ###### Execution ###### 
    execution_results = env_interface.execute_plans(plans)
    execution_success = execution_results[-1]
    if not execution_success:
        return 'EXECUTION_FAIL'
    
    return 'SUCCESS'



def pick(pre_pick_pose: SE3, pick_pose: SE3, post_pick_pose: SE3, env_interface: EdfRosInterface) -> str:

    env_interface.release()

    ###### Pre-pick Plan ######
    plan_targets = [(pre_pick_pose, "default", {}),
                    (pick_pose, "cartesian", {'cartesian_step': 0.01, 'cspace_step_thr': 10., 'avoid_collision': False, 'success_fraction': 0.95}),
                   ]
    plan_results, plans = env_interface.move_plans(targets=plan_targets)
    plan_success = plan_results[-1]
    if not plan_success:
        return 'PRE_PICK_PLAN_FAIL'



    ###### Pre-pick Execution ######
    execution_results = env_interface.execute_plans(plans)
    execution_success = execution_results[-1]
    if not execution_success:
        return 'PRE_PICK_EXECUTION_FAIL'


    ###### Grasp ######
    grasp_result = env_interface.grasp()
    if not grasp_result:
        return 'GRASP_FAIL'


    ###### Post-pick Plan ######
    plan_targets = [(post_pick_pose, "cartesian", {'cartesian_step': 0.01, 'cspace_step_thr': 10., 'avoid_collision': False, 'success_fraction': 0.95}),
                    ]
    plan_results, plans = env_interface.move_plans(targets=plan_targets)
    plan_success = plan_results[-1]
    if not plan_success:
        return 'POST_PICK_PLAN_FAIL'
    

    ###### Post-pick Execution ######
    execution_results = env_interface.execute_plans(plans)
    execution_success = execution_results[-1]
    if not execution_success:
        return 'POST_PICK_EXECUTION_FAIL'
    
    return 'SUCCESS'
    
def place(pre_place_pose: SE3, place_pose: SE3, post_place_pose: SE3, env_interface: EdfRosInterface) -> str:

    ###### Pre-place Plan ######
    plan_targets = [(pre_place_pose, "default", {}),
                    (place_pose, "cartesian", {'cartesian_step': 0.01, 'cspace_step_thr': 10., 'avoid_collision': False, 'success_fraction': 0.95}),
                   ]
    plan_results, plans = env_interface.move_plans(targets=plan_targets)
    plan_success = plan_results[-1]
    if not plan_success:
        return 'PRE_PLACE_PLAN_FAIL'

    ###### Pre-place Execution ######
    execution_results = env_interface.execute_plans(plans)
    execution_success = execution_results[-1]
    if not execution_success:
        return 'PRE_PLACE_EXECUTION_FAIL'




    ###### Release ######
    env_interface.detach()
    release_result = env_interface.release()
    if not release_result:
        return 'RELEASE_FAIL'




    ###### Post-place Plan ######
    plan_targets = [(post_place_pose, "cartesian", {'cartesian_step': 0.01, 'cspace_step_thr': 10., 'avoid_collision': False, 'success_fraction': 0.95}),
                   ]
    plan_results, plans = env_interface.move_plans(targets=plan_targets)
    plan_success = plan_results[-1]
    if not plan_success:
        return 'POST_PLACE_PLAN_FAIL'
    
    ###### Post-place Execution ######
    execution_results = env_interface.execute_plans(plans)
    execution_success = execution_results[-1]
    if not execution_success:
        return 'POST_PLACE_EXECUTION_FAIL'
    
    return 'SUCCESS'

### Initialize EDF ROS Interface

In [3]:
env_interface = EdfRosInterface(reference_frame = "scene")
env_interface.reset()
env_interface.moveit_interface.arm_group.set_planning_time(seconds=5)

[ WARN] [1676289490.895626488]: Link axle_link has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1676289490.895641839]: Link front_rocker_link has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1676289490.895653548]: Link rear_rocker_link has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1676289490.895663526]: Link front_cover_link has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1676289490.895668953]: Link front_lights_link has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying colli

[INFO] [1676289492.299922]: Commencing scene point cloud update...
[INFO] [1676289492.750516]: Processing received scene point cloud...
[INFO] [1676289492.784966]: Scene pointcloud update success!
[INFO] [1676289492.785648]: Commencing end-effector point cloud update...
[INFO] [1676289493.236235]: Processing received end-effector point cloud...
[INFO] [1676289493.237026]: End-effector pointcloud update success!


### Initialize dataset list

In [4]:
dataset = []

### Observe Point clouds

In [5]:
grasp_raw = env_interface.observe_eef(obs_type = 'pointcloud', update = True)
scene_raw = env_interface.observe_scene(obs_type = 'pointcloud', update = True)

[INFO] [1676289497.954196]: Commencing end-effector point cloud update...
[INFO] [1676289499.389419]: Processing received end-effector point cloud...
[INFO] [1676289499.390275]: End-effector pointcloud update success!
[INFO] [1676289499.390906]: Commencing scene point cloud update...
[INFO] [1676289500.336350]: Processing received scene point cloud...
[INFO] [1676289500.341287]: Scene pointcloud update success!


### Pick

In [6]:
pick_poses = SE3([0.0, 0.0, 1.0, 0.0, -0.05, 0.0, 0.275])
pre_pick_poses = SE3.multiply(pick_poses, SE3([1., 0., 0., 0., 0., 0., -0.1]))
post_pick_poses = SE3.multiply(SE3([1., 0., 0., 0., 0., 0., 0.2]), pick_poses)

In [7]:
idx = 0
pick_pose, pre_pick_pose, post_pick_pose = pick_poses[idx], pre_pick_poses[idx], post_pick_poses[idx]

In [8]:
pick_result = pick(pre_pick_pose, pick_pose, post_pick_pose, env_interface=env_interface)
print(pick_result)

[INFO] [1676289501.134312]: EDF Moveit Interface: Found a motion plan with length: 9
[INFO] [1676289501.149547]: EDF Moveit Interface: Found a Cartesian plan with length: 12
[INFO] [1676289503.860097]: EDF Moveit Interface: Found a Cartesian plan with length: 21
SUCCESS


In [9]:
assert pick_result == 'SUCCESS'
pick_demo = TargetPoseDemo(target_poses=pick_poses, scene_pc=scene_raw, grasp_pc=grasp_raw)

In [10]:
env_interface.detach()
env_interface.attach_sphere(radius=0.1, pos=[0,0,0.22]) # To avoid collsion with mug

### Observe gripper

In [11]:
result = move_simple(target_poses = SE3([0., 0., 1., 0., -0.1, 0., 0.6]), env_interface=env_interface)
print(result)

# result = move_simple(target_poses = SE3([[0., 0., 1., 0., -0.1, 0., 0.6], 
#                                          [0.5, -0.5, -0.5, -0.5, 0.0, 0., 0.5], 
#                                          [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5]]), 
#                      env_interface=env_interface)
# print(result)

[INFO] [1676289504.825269]: EDF Moveit Interface: Found a motion plan with length: 4
SUCCESS


In [12]:
if result == 'SUCCESS':
    env_interface.detach()
    grasp_raw = env_interface.observe_eef(obs_type = 'pointcloud', update = True)
    env_interface.attach(pcd = grasp_raw)

[INFO] [1676289505.601746]: Commencing end-effector point cloud update...
[INFO] [1676289506.917661]: Processing received end-effector point cloud...
[INFO] [1676289506.918668]: End-effector pointcloud update success!


### Observe Scene

In [13]:
result = move_simple(target_poses = SE3([0.0, 0.0, 1.0, 0.0, -0.4, 0.4, 0.6]), env_interface=env_interface)
print(result)

[INFO] [1676289507.738211]: EDF Moveit Interface: Found a motion plan with length: 15
SUCCESS


In [14]:
if result == 'SUCCESS':
    scene_raw = env_interface.observe_scene(obs_type = 'pointcloud', update = True)

[INFO] [1676289510.116105]: Commencing scene point cloud update...
[INFO] [1676289511.312009]: Processing received scene point cloud...
[INFO] [1676289511.316335]: Scene pointcloud update success!


In [15]:
result = move_simple(target_poses = SE3([0., 0., 1., 0., -0.1, 0., 0.6]), env_interface=env_interface)
print(result)

[INFO] [1676289523.247147]: EDF Moveit Interface: Found a motion plan with length: 23
SUCCESS


### Place

In [16]:
place_poses = SE3([0.5000, -0.5000, -0.5000, -0.5000, 0.12, -0.18, 0.30])
_, pre_place_poses = optimize_pcd_collision(x=scene_raw, y=grasp_raw, 
                                            cutoff_r = 0.03, dt=0.01, eps=1., iters=5,
                                            rel_pose=place_poses)
post_place_poses = place_poses * pick_pose.inv() * pre_pick_pose

In [17]:
idx = 0
place_pose, pre_place_pose, post_place_pose = place_poses[idx], pre_place_poses[idx], post_place_poses[idx]

In [18]:
result = place(pre_place_pose, place_pose, post_place_pose, env_interface)
print(result)

[INFO] [1676289526.027829]: EDF Moveit Interface: Found a motion plan with length: 20
[INFO] [1676289526.095209]: EDF Moveit Interface: Found a Cartesian plan with length: 6
[INFO] [1676289530.831754]: EDF Moveit Interface: Found a Cartesian plan with length: 11
SUCCESS


In [19]:
assert result == 'SUCCESS'
place_demo = TargetPoseDemo(target_poses=place_poses, scene_pc=scene_raw, grasp_pc=grasp_raw)

### Save

In [20]:
demo_seq = DemoSequence(demo_seq = [pick_demo, place_demo])
dataset.append(demo_seq)
save_demos(demos=dataset, dir="demo/test_demo")